# Preparation du tableau de données pour les analyses statistiques

## Choix du dossier de travail

In [1]:
# Donner le chemin vers le dossier de travail choisi
setwd("shared/projects/sorbonne_master_ue_projet/Colaris")

ERROR: Error in setwd("shared/projects/sorbonne_master_ue_projet/Colaris"): cannot change working directory


## Installation des packages

In [18]:
# Installation du package qiime2R depuis github s'il n'est pas déjà présent
if (!requireNamespace("devtools", quietly = TRUE)){install.packages("devtools")}
devtools::install_github("jbisanz/qiime2R")

Installing package into ‘/shared/home/mthemezeleroy/R/x86_64-conda-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)




fs           (1.6.4     -> 1.6.5    ) [CRAN]
withr        (3.0.1     -> 3.0.2    ) [CRAN]
yulab.utils  (0.1.7     -> 0.1.8    ) [CRAN]
Rcpp         (1.0.13    -> 1.0.13-1 ) [CRAN]
bitops       (1.0-8     -> 1.0-9    ) [CRAN]
RCurl        (1.98-1.14 -> 1.98-1.16) [CRAN]
xfun         (0.48      -> 0.49     ) [CRAN]
evaluate     (1.0.0     -> 1.0.1    ) [CRAN]
tinytex      (0.53      -> 0.54     ) [CRAN]
knitr        (1.48      -> 1.49     ) [CRAN]
fontawesome  (0.5.2     -> 0.5.3    ) [CRAN]
gtable       (0.3.5     -> 0.3.6    ) [CRAN]
rmarkdown    (2.28      -> 2.29     ) [CRAN]
rstudioapi   (0.16.0    -> 0.17.1   ) [CRAN]
checkmate    (2.3.1     -> 2.3.2    ) [CRAN]
RcppArmad... (14.0.2-1  -> 14.2.0-1 ) [CRAN]
later        (1.3.2     -> 1.4.1    ) [CRAN]
promises     (1.3.0     -> 1.3.2    ) [CRAN]
htmlTable    (2.4.2     -> 2.4.3    ) [CRAN]
data.table   (1.16.0    -> 1.16.2   ) [CRAN]
igraph       (2.0.3     -> 2.1.1    ) [CRAN]
NADA         (NA        -> 1.6-1.1  ) [CRAN]
Hmisc     

Skipping 23 packages ahead of CRAN: S4Vectors, BiocGenerics, zlibbioc, XVector, GenomeInfoDbData, SparseArray, DelayedArray, S4Arrays, GenomeInfoDb, IRanges, GenomicRanges, MatrixGenerics, SummarizedExperiment, rhdf5filters, Rhdf5lib, rhdf5, treeio, BiocParallel, Biostrings, SingleCellExperiment, multtest, biomformat, TreeSummarizedExperiment

Installing 24 packages: fs, withr, yulab.utils, Rcpp, bitops, RCurl, xfun, evaluate, tinytex, knitr, fontawesome, gtable, rmarkdown, rstudioapi, checkmate, RcppArmadillo, later, promises, htmlTable, data.table, igraph, NADA, Hmisc, zCompositions

Installing packages into ‘/shared/home/mthemezeleroy/R/x86_64-conda-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘fs’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘Rcpp’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘bitops’ had non-zero exit status”
Warning message in i.p(...):

── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/slurm_mthemezeleroy_36998225.tmp/Rtmp5V4XhZ/remotes2d9102b2d5646/jbisanz-qiime2R-d0610e1/DESCRIPTION’ ... OK
* preparing ‘qiime2R’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘qiime2R_0.99.6.tar.gz’



Installing package into ‘/shared/home/mthemezeleroy/R/x86_64-conda-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)



### Chargement des librairies

In [18]:
library(qiime2R)
library(tidyverse)

### Importation des données

#### Metadonnées de l'expérience

In [19]:
# Importation des meta données associées à l'expérience sur la température et le nourrissage de 4 espèces de nématodes
meta_data <-read.table("data/meta-data2.tsv", header= T)

#### Table de contingence des ASVs

In [20]:
# Importation de la table de contingence des ASVs de chaque échantillon
ASV <- read_qza("data/table-nematodesAM.qza")
ASV_tbl <- as.data.frame(ASV$data) # création d'une table comprenant uniquement les données sur les ASVs et leur nombre
ASV_tbl <- as.data.frame(t(ASV_tbl)) # transposer le dataframe pour avoir un échantillon par ligne et les ASVs en colonne
ASV_tbl <- rownames_to_column(ASV_tbl, var = "sample.id") # transformer le nom des lignes en une colonne intitulée "sample.id"

#### Table des assignations taxonomiques en fonction des bases de référence

In [23]:
assign_RDP <- tibble(read.csv("data/output/assign_RDP.csv",sep = ";"))
names(assign_RDP)[9] <- "base_ref"
assign_SILVA <- tibble(read.csv("data/output/assign_SILVA.csv",sep = ";"))
assign_GG2 <- tibble(read.csv("data/output/assign_GG2.csv",sep = ";"))

# Regroupper les trois tables
assign_3ref <- bind_rows(assign_RDP, assign_SILVA, assign_GG2)

### Fusion des dataframes

In [26]:
# Fusionner la table de metadonnées et la table de contingence des ASVs en fonction de l'échantillon
PM4_data <- merge(ASV_tbl,meta_data, by= "sample.id") |> filter(species== "PM4" & food == "with_food")|> select(-c('species','food','replicats','treatment','description'))

# Pivoter le tableau resultant en version longue pour avoir un ASV par ligne pour tous les échantillons
PM4_data_lg <- pivot_longer(PM4_data, cols=-c(sample.id,temperature),names_to ="ASV",values_to ="frequence") |> 
    filter(frequence > 0) # filtrer pour ne garder que les lignes pour lesquelles la fréquence de l'ASV est supérieure à 0 

In [28]:
# Fusionner PM4_data_lg avec les assignations
data_final_pm4 <- merge(PM4_data_lg,assign_3ref, by= "ASV")

## Sauvegarde des jeux de données crées

In [29]:
# Sauvegarder la fusion des métadonnées et des contingence d'ASVs : PM4_data_lg
write.table(PM4_data_lg, file = "data/output/PM4_data_clean.csv", sep = ";",row.names = FALSE,col.names = TRUE)
# Sauvegarder le tableau de données final : data_final  
write.table(data_final_pm4, file = "data/output/data_final_pm4.csv", sep = ";", row.names = FALSE, col.names = TRUE)